In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Intellectual_Disability/GSE192767'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression data from human lymphoblastoid cell lines (LCLs)"
!Series_summary	"The X-linked alpha thalassaemia intellectual disability syndrome (ATRX) protein is a member of the SWI/SNF family of chromatin remodelling factors which acts as an ATP dependent molecular motor. Germline mutations in ATRX give rise to a severe form of syndromal intellectual disability (ATR-X syndrome). To date, only a small number of genes have been identified that are affected by pathogenic ATRX mutations in human."
!Series_summary	"We performed microarray experiments on LCLs  from normal individuals and  patients with diverse pathogenic ATRX mutations, to identify more genes regulated by ATRX."
!Series_overall_design	"We used 20 LCLs  from unaffected individuals and  28 LCLs from patients for RNA extraction and hybridization on Affymetrix microarrays."
Sample Characteristics Dictionary:
{0: ['phenotype: ATR-X syndrome', 'phenotype: unaffected'], 1: ['cell type: human l

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Based on the background information, it is likely that the dataset contains gene expression data.
is_gene_available = True

# Checking the sample characteristics dictionary for variable availability
sample_characteristics = {
    0: ['phenotype: ATR-X syndrome', 'phenotype: unaffected'],
    1: ['cell type: human lymphoblastoid cell line (LCL)']
}

# Intellectual_Disability data availability
# Variable "phenotype" could be used to infer Intellectual_Disability
if 'phenotype: ATR-X syndrome' in sample_characteristics[0] and 'phenotype: unaffected' in sample_characteristics[0]:
    trait_row = 0
else:
    trait_row = None

# Age and gender data is not available in the given sample characteristics dictionary
age_row = None
gender_row = None

# Data Type Conversion Functions
def convert_trait(value):
    if 'ATR-X syndrome' in value:
        return 1
    elif 'unaffected' in value:
        return 0
    return None

convert_age = lambda value: None  # Function to convert age values if available (not applicable here)
convert_gender = lambda value: None  # Function to convert gender values if available (not applicable here)

# Saving cohort information
save_cohort_info('GSE192767', './preprocessed/Intellectual_Disability/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction if trait_row is available
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Intellectual_Disability', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Intellectual_Disability/trait_data/GSE192767.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM5765052': [1], 'GSM5765053': [1], 'GSM5765054': [1], 'GSM5765055': [1], 'GSM5765056': [1], 'GSM5765057': [1], 'GSM5765058': [1], 'GSM5765059': [1], 'GSM5765060': [1], 'GSM5765061': [1], 'GSM5765062': [1], 'GSM5765063': [1], 'GSM5765064': [1], 'GSM5765065': [1], 'GSM5765066': [1], 'GSM5765067': [1], 'GSM5765068': [1], 'GSM5765069': [1], 'GSM5765070': [1], 'GSM5765071': [1], 'GSM5765072': [1], 'GSM5765073': [1], 'GSM5765074': [1], 'GSM5765075': [1], 'GSM5765076': [1], 'GSM5765077': [1], 'GSM5765078': [1], 'GSM5765079': [1], 'GSM5765080': [0], 'GSM5765081': [0], 'GSM5765082': [0], 'GSM5765083': [0], 'GSM5765084': [0], 'GSM5765085': [0], 'GSM5765086': [0], 'GSM5765087': [0], 'GSM5765088': [0], 'GSM5765089': [0], 'GSM5765090': [0], 'GSM5765091': [0], 'GSM5765092': [0], 'GSM5765093': [0], 'GSM5765094': [0], 'GSM5765095': [0], 'GSM5765096': [0], 'GSM5765097': [0], 'GSM5765098': [0], 'GSM5765099': [0]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at',
       '11715104_s_at', '11715105_at', '11715106_x_at', '11715107_s_at',
       '11715108_x_at', '11715109_at', '11715110_at', '11715111_s_at',
       '11715112_at', '11715113_x_at', '11715114_x_at', '11715115_s_at',
       '11715116_s_at', '11715117_x_at', '11715118_s_at', '11715119_s_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at', '11715104_s_at'], 'GeneChip Array': ['Human Genome PrimeView Array', 'Human Genome PrimeView Array', 'Human Genome PrimeView Array', 'Human Genome PrimeView Array', 'Human Genome PrimeView Array'], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['30-Mar-16', '30-Mar-16', '30-Mar-16', '30-Mar-16', '30-Mar-16'], 'Sequence Type': ['Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database'], 'Transcript ID(Array Design)': ['g21264570', 'g21264570', 'g21264570', 'g22748780', 'g30039713'], 'Target Description': ['g21264570 /TID=g21264570 /CNT=1 /FEA=FLmRNA /TIER=FL /STK=0 /

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Define the keys for identifier and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'Gene Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)
print(preview_df(mapping_df, n=5)) # Previewing the first 5 rows of the mapping dataframe

# 3. Apply the mapping to get the gene expression dataframe
gene_data = apply_gene_mapping(gene_data, mapping_df)


{'ID': ['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at', '11715104_s_at'], 'Gene': ['HIST1H3G', 'HIST1H3G', 'HIST1H3G', 'TNFAIP8L1', 'OTOP2']}


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Intellectual_Disability/gene_data/GSE192767.csv'
normalized_gene_data.to_csv(gene_csv_path)
print(preview_df(normalized_gene_data, n=5))  # Optional but recommended for inspection

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)
print(preview_df(merged_data, n=5))  # Optional but recommended for inspection

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Intellectual_Disability')

# 4. Save the cohort information.
save_cohort_info('GSE192767', './preprocessed/Intellectual_Disability/cohort_info.json', True, True, trait_biased, merged_data)

# 5. If the trait is not severely biased, save the merged data to a csv file.
if not trait_biased:
    csv_path = './preprocessed/Intellectual_Disability/GSE192767.csv'
    unbiased_merged_data.to_csv(csv_path)


{'GSM5765052': [5.79996214940397, 3.9928688006619604, 3.95521711983455, 3.40047980022129, 3.29673722783375], 'GSM5765053': [5.70050934220473, 4.07252329716756, 3.97028916746449, 3.208495575441095, 3.33718758499198], 'GSM5765054': [5.77833308521097, 4.13076382465344, 4.77559913915191, 3.369652719118225, 3.42574620558311], 'GSM5765055': [5.60298546993569, 4.151200736139545, 4.10895739292915, 3.3658175637776653, 3.27853557754664], 'GSM5765056': [5.98648796769992, 3.951338377992535, 4.53228126790821, 3.33925895442437, 3.1963218146276], 'GSM5765057': [6.02190383766546, 4.181437772720249, 4.18080357398925, 3.26812864915946, 3.47395892202044], 'GSM5765058': [5.79830063254858, 4.297868442494066, 4.12401637916933, 3.48158669225541, 3.39024123509017], 'GSM5765059': [5.70334249096542, 4.145113788720235, 4.14085366737952, 3.560155517987435, 3.46990904073522], 'GSM5765060': [5.9680386667567, 4.48531099828687, 4.19337325854738, 3.5925953138596753, 3.45520825156798], 'GSM5765061': [5.66025676073873, 

{'Intellectual_Disability': [1.0, 1.0, 1.0, 1.0, 1.0], 'A1BG': [5.79996214940397, 5.70050934220473, 5.77833308521097, 5.60298546993569, 5.98648796769992], 'A1CF': [3.9928688006619604, 4.07252329716756, 4.13076382465344, 4.151200736139545, 3.951338377992535], 'A2M': [3.95521711983455, 3.97028916746449, 4.77559913915191, 4.10895739292915, 4.53228126790821], 'A2ML1': [3.40047980022129, 3.208495575441095, 3.369652719118225, 3.3658175637776653, 3.33925895442437], 'A3GALT2': [3.29673722783375, 3.33718758499198, 3.42574620558311, 3.27853557754664, 3.1963218146276], 'A4GALT': [4.793995580103137, 4.766702057539404, 4.849046890472359, 4.7803275270062, 4.668519272552261], 'A4GNT': [4.78821538737212, 4.88957632212195, 4.92507707575226, 4.90716185153236, 4.67037867756614], 'AAAS': [7.84366509765241, 7.8418023455281, 7.69043643464455, 7.58920567366436, 7.73329324384804], 'AACS': [7.97294058727831, 7.679805883211888, 7.794388056145451, 7.976944634123315, 7.837758145066215], 'AACSP1': [4.0483022115032